In [2]:
import random as rnd
import pyglet
import numpy
import time
"""

Creado por: Daniel Germán Martínez 
Universidad Nacional de Colombia

"""

'\n\nCreado por: Daniel Germán Martínez \nUniversidad Nacional de Colombia\n\n'

In [3]:
class Covid:

    def __init__(self, window_width,windo_height , cell_size):
        self.grind_width  = 100 #Tamaño de la malla 
        self.grind_height = 100
        self.cell_size    = cell_size #Tamaño celula
        self.cells        = []
        self.generate_cells()

#Defino el alfabato -> Estados que puede tener la celula 
#Np  = Celda Vacia                                  = 0 = Negro
#Ps  = Persona sana                                 = 1 = Blanco 
#Cs  = Persona Covid sintomatica                    = 2 = Roja 
#Cns = Persona Covid asintomaticca                  = 3 = Verde
#Ct  = Persona Covid en cuarentena o en el hospital = 4 = Azul 
#Pac = Persona curada con anticuerpos               = 5 = Celeste

    def generate_cells(self): #Genera la matrz
        for row in range(0, 100): 
            self.cells.append([])
            for col in range(0, 100):
                self.cells[row].append([0,0])

        for i in range(5000):
            r1 = rnd.randint(0, 99)
            c1 = rnd.randint(0, 99)
            while self.cells[r1][c1][0] == 1:     #Ubico personas sanas aleatoriamente ... no se si se deben uicar de alguna manera en especial.
                r1 = rnd.randint(0, 99)
                c1 = rnd.randint(0, 99)
            self.cells[r1][c1][0] = 1

        for i in range (0,5):
            cond2 = True
            while cond2 == True:
                r1 = rnd.randint(0, 99)
                c1 = rnd.randint(0, 99)
                if self.cells[r1][c1][0] == 1 or self.cells[r1][c1][0] == 2:
                    r1 = rnd.randint(0, 99)
                    c1 = rnd.randint(0, 99)   #5 Personas Sintomaticas agregadas aleatoriamente 
                else:
                    self.cells[r1][c1][0] = 2 
                    cond2 = False
            cond2 = True
        for i in range (0,5):
            cond3 = True
            while cond3 == True:
                r1 = rnd.randint(0, 99)
                c1 = rnd.randint(0, 99)
                if self.cells[r1][c1][0] == 1 or self.cells[r1][c1][0] == 2 or self.cells[r1][c1] == 3:
                    r1 = rnd.randint(0, 99)   #5 Personas Asintomaticas agregadas aleatoriamente
                    c1 = rnd.randint(0, 99)
                else:
                    self.cells[r1][c1][0] = 3
                    cond3 = False
            cond3 = True

    def run_rules(self):
        temp = [] #Creacion de matriz auxiliar
        for row in range(0, self.grind_height): 
            temp.append([])
            for col in range(0, self.grind_width):
                temp[row].append([0,0])

        for row in range(0, self.grind_height ):
            for col in range(0, self.grind_width ):
###########################################################################################################################                
                if self.cells[row][col][0]==1 :  
                    sum_veci = sum([self.get_cell_value_Ps(row - 1,col    ),
                                    self.get_cell_value_Ps(row - 1,col - 1),
                                    self.get_cell_value_Ps(row    ,col - 1),
                                    self.get_cell_value_Ps(row + 1,col - 1),
                                    self.get_cell_value_Ps(row + 1,col    ),
                                    self.get_cell_value_Ps(row + 1,col + 1),
                                    self.get_cell_value_Ps(row    ,col + 1),
                                    self.get_cell_value_Ps(row - 1,col + 1)])
                    prob = sum_veci/10
                    opciones = [1, 0]
                    probabilidades = [prob, (1-prob)]
                    eleccion = numpy.random.choice(opciones, p=probabilidades)
                    if eleccion ==1:
                        opciones2 = [2, 3]
                        probabilidades2 = [0.3, 0.7]
                        eleccion2 = numpy.random.choice(opciones2, p=probabilidades2)
                        temp[row][col][0] = eleccion2
                    else:
                        x = self.cells[row][col][1]
                        x +=1                        
                        temp[row][col] = [1 ,x]
##################################################################################################################################
                elif self.cells[row][col][0] == 2:
                    if self.cells[row][col][1] >= 7:
                        temp[row][col][0] = 4   
                    else: ####
                        x = self.cells[row][col][1]
                        x +=1
                        temp[row][col] = [2 , x]
#####################################################################################################################

                elif self.cells[row][col][0] == 3:
                    x = self.cells[row][col]#Persona Cns
                    if x[1] >= 30:
                        temp[row][col][0] = 5  
                    else: ####
                        x = self.cells[row][col][1]
                        x +=1
                        temp[row][col] = [3 , x]
 ####################################################################################################################
                elif self.cells[row][col][0] == 4:#Persona Ct
                    if self.cells[row][col][1] >= 20:
                        opcion4 = [1,2]
                        probabilidades4 = [0.8,0.2]
                        eleccion4 = numpy.random.choice(opcion4,p=probabilidades4)
                        if eleccion4 == 1:
                            temp[row][col][0] = 5
                        elif eleccion4 == 2:
                            temp[row][col][0] = 0                            
                    else: 
                        x = self.cells[row][col][1]
                        x +=1
                        temp[row][col] = [4 , x]
                        
#####################################################################################################################                
                elif self.cells[row][col][0] == 5:#Persona Pac
                    x = self.cells[row][col][1]
                    x +=1
                    
                    temp[row][col] = [self.cells[row][col][0] , x]
#####################################################################################################################                
## Movimiento de las celulas
        for row in range(0, self.grind_height ):
            for col in range(0, self.grind_width ):
                L = self.direccion_mov(row , col , temp)
                if L != [row,col]:
                    if temp[row][col][0] == 4 or temp[row][col][0] == 0:
                        temp[row][col] = temp[row][col]
                    else: 
                        temp[L[0]][L[1]] = temp[row][col]
                        temp[row][col] = [0,0]
                
        self.cells = temp
        temp = []

    def direccion_mov(self,row,col,temp): #Elige la direccion del movimiento y se reproduce de manera secuencial
        opcion = [1,2,3,4]
        probabilidades = [0.25,0.25,0.25,0.25]
        eleccion = numpy.random.choice(opcion,p=probabilidades)

        if eleccion == 1: #Sur
            if  row + 1> self.grind_height - 1 :
                if temp[0][col][0] != 0:
                    return [row,col]
                else:
                    return [0,col]
            else:
                if temp[row + 1][col][0] != 0:
                    return [row,col]
                else:
                    return [row + 1,col]

        elif eleccion == 2: #Norte
            if  row - 1 < 0 :
                if  temp[self.grind_height - 1 ][col][0] != 0:
                    return [row,col]
                else:
                    return[self.grind_height - 1,col]
            else:
                if  temp[row - 1][col][0] != 0:
                    return [row,col]
                else:
                    return [row - 1,col]
                    
        elif eleccion == 3: # Derecha
            if  col + 1 > self.grind_width - 1 :
                if  temp[row][0][0] !=0:
                    return [row,col]
                else:
                    return [row,0]
            else:
                if  temp[row][col + 1][0] != 0:
                    return [row,col]
                else:
                    return [row,col + 1]
            
        elif eleccion == 4: #Izquierda
            if  col - 1 < 0 :
                if  temp[row][self.grind_width - 1][0]!= 0:
                    return [row,col]
                else:
                    return[row,self.grind_width - 1]
            else:
                if  temp[row][col - 1][0]!= 0:
                    return [row,col]
                else:
                    return [row,col - 1]
        else: 
            print("Error")
    def contador(self): #Funcion para mantener conteo de cada estado de las celulas.
        Lista_contadores = [0 , 0 , 0 , 0 , 0 ]
        for row in range(0, self.grind_height):
            for col in range(0, self.grind_width):
                valor_celda = { "1" : 0,
                                "2" : 1,
                                "3" : 2,
                                "4" : 3,
                                "5" : 4
                              }
                if self.cells[row][col][0] == 0:
                    pass
                else:
                    t = valor_celda[str(self.cells[row][col][0])]
                    Lista_contadores[t] +=1
        return Lista_contadores
                
    def get_cell_value_Ps(self,row,col): 
        if row >= 0 and row<self.grind_height and col >= 0 and col < self.grind_width and self.cells[row][col][0] != 0: #Obtiene el valor de cada celula
            return 1

        elif row < 0 and col >= 0 and col < self.grind_width: #Se sale por arriba de la matriz
            if self.cells[self.grind_height  - 1][col][0] != 0:
                return 1
            else:
                return 0

        elif row >= 0 and col< 0 and row<self.grind_height: #Se sale por la izquierda
            if self.cells[row][self.grind_width - 1][0] != 0:
                return 1
            else:
                return 0

        elif row >= 0 and col>= self.grind_width and row<self.grind_height: #Se sale por la derecha
            if self.cells[row][0][0] != 0:
                return 1
            else:
                return 0

        elif row >= self.grind_height and col>= 0 and col < self.grind_width: #Se sale por abajo
            if self.cells[0][col][0] != 0:
                return 1
            else:
                return 0
        elif row < 0 and col< 0: #Esquina superior derecha
            if self.cells[self.grind_height  - 1][self.grind_width - 1][0] != 0:
                return 1
            else:
                return 0

        elif row >= self.grind_height and col>= self.grind_width : #Esquina inferior izquierda 
            if self.cells[0][0][0] != 0:
                return 1
            else:
                return 0

        elif row < 0 and col>= self.grind_width: #Esquina superior izquierda
            if self.cells[self.grind_height  - 1][self.grind_width - 1][0] != 0:
                return 1
            else:
                return 0

        elif row >= self.grind_height and col< 0 : #Esquina inferior derecha
            if self.cells[0][0][0] != 0:
                return 1
            else:
                return 0
        else:
            return 0 

    def draw(self): #Dibuja la AC en pantalla
        for row in range(0, self.grind_height):
            for col in range(0, self.grind_width):
                color_celda = { "0" : (0 , 0 , 0),#colores
                                "1" : (255,255,255),
                                "2" : (255,0,0),
                                "3" : (0,255,0),
                                "4" : (0,0,255),
                                "5" : (102,178,255)
                              }
                square_coords = (row * self.cell_size , col * self.cell_size,
                                row * self.cell_size , col * self.cell_size + self.cell_size,
                                row * self.cell_size + self.cell_size , col * self.cell_size,
                                row * self.cell_size + self.cell_size , col * self.cell_size + self.cell_size)
                colorpairs = color_celda[str(self.cells[row][col][0])] * int(len(square_coords)/2) #Colorpair representa que cada dos coordenadas van a tener el color asignado por el codigo decimal (R,G,B)
                pyglet.graphics.draw_indexed(4,pyglet.gl.GL_TRIANGLES,
                                            [0,1,2,1,2,3],
                                            ('v2i',(square_coords)),
                                            ('c3B', colorpairs)) 

    def drawfin(self,ciclos,vivos,muertos):
        te  = "Se estabilizo el sistema a los {} ciclos, con {} vivos y {} muertos" .format(ciclos,vivos,muertos)
        label = pyglet.text.Label(te ,
                          font_name='Times New Roman',
                          font_size=10,
                          x=250,
                          y=250,
                          anchor_x='center', anchor_y='center')
        label.draw()
        
                                       

In [4]:
import pyglet
#from reglas_covid import Covid  
import time

class Window(pyglet.window.Window):

    def __init__(self):
        super().__init__(500,500)# Con este tamaño se generan 10.000 celulas AC100x100
        self.reglas_simulacion_covid = Covid(self.get_size()[0], 
                                    self.get_size()[1],  
                                     5)
        self.ciclos = 0
        self.conteo = self.reglas_simulacion_covid.contador()
        self.file = open("Datos.txt", "w") #Arhicvo que almacena los datos generados
        x = self.ciclos
        su = sum(self.conteo)
        imprimir = "En el ciclo {},\n hay {} personas sanas \n {} personas sintomatica \n {} personas asintomatica \n {} personas cuarentena \n {} personas anticuerpo \n total {}".format(x,self.conteo[0],self.conteo[1],self.conteo[2],self.conteo[3],self.conteo[4],su)
        self.file.write(imprimir + "\n")
        pyglet.clock.schedule_interval(self.update,10/20)
        self.count = 0
        
    def on_draw(self):
        self.clear()
        if self.conteo[0] == 0 and self.conteo[1] == 0 and self.conteo[2] == 0 and self.conteo[3] == 0 :
            if self.count == 0:
                pyglet.clock.unschedule(self.update)
                time.sleep(2)
                X = 5010 - self.conteo[4]
                self.reglas_simulacion_covid.drawfin(self.ciclos , self.conteo[4] , X)
                self.count += 1
                self.file.write("Hubo {} muertos y {} sobrevivieron, es decir un {} porciento murio ".format(X , self.conteo[4] , X/5010 ))
                
            elif self.count == 1:
                time.sleep(15)
                self.file.close() #una vez acabado el programa despues de 15 sec se cierra
                pyglet.app.exit()     
        else:
            self.reglas_simulacion_covid.draw()
    def update(self, dt):
        self.reglas_simulacion_covid.run_rules()
        self.conteo = self.reglas_simulacion_covid.contador()
        self.ciclos += 1
        x = self.ciclos
        su = sum(self.conteo)
        imprimir = "En el ciclo {},\n hay {} personas sanas \n {} personas sintomatica \n {} personas asintomatica \n {} personas cuarentena \n {} personas anticuerpo \n total {}".format(x,self.conteo[0],
        self.conteo[1],self.conteo[2],self.conteo[3],self.conteo[4],su)
        self.file.write(imprimir + "\n")

if __name__ =='__main__':
    window = Window()
    pyglet.app.run() 